In [1]:
# import cProfile
# import collections
import datetime

import os
import pickle
# import sys
import time

# import numpy as np
import pandas as pd

# from datetime import datetime
from dotenv import load_dotenv

pd.options.display.float_format = "{:.2f}".format
# plt.rcParams.update({'font.size': 18})

In [2]:
load_dotenv()
SPOTIPY_API = os.getenv('SPOTIPY_API')
SPOTIPY_API_KEY = os.getenv('SPOTIPY_API_KEY')
scope=''

# https://spotipy.readthedocs.io/en/latest/#
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_API, client_secret=SPOTIPY_API_KEY)

# help(sp)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager,requests_timeout=300)

# 1. Get user and playlist IDs

In [3]:
n = 950 # max = 950
p_limit = 50 # limit of playlists max = 50
years = range(2018,2022,1)

In [ ]:
# Extract the user and playlist IDs
format = '%d/%m/%Y %H:%M'
t = time.time()

# check if data extraction already exists then load if it exists, otherwise performs the data request from API and save the file
data_path = os.getcwd() + '\\data\\external\\raw_1_uid_pid.pkl'
if os.path.exists(data_path):
    print('Playlist ID and User ID - Data already requested from API - Loaded: ' + data_path)
    # load file only if the next step was not executed (getting tracks)
    data_path_2 = os.getcwd() + '\\data\\external\\raw_2_playlists.pkl'
    if not(os.path.exists(data_path_2)):
        with open(data_path, 'rb') as f:
            uid_pid_raw = pickle.load(f)
else:
    print('Playlist ID and User ID - Data requested from API - Saved: ' + data_path)
    # list of all user ids (UID) and playlist ids (PID)
    uid_pid_raw = []
    
    for y in years:
        txt = 'year:' + str(y)
        print(txt)
        for i in range(0,n,1):
            if i % 100 == 0:
                t_now = datetime.datetime.now().strftime(format)
                calc_eta = datetime.datetime.now() + datetime.timedelta(seconds = n - i) * (time.time() - t) / (i + 1)
                calc_eta = calc_eta.strftime(format)
                print("%s - %d/%d elapsed time (min): %.1f - ETA: %s" % (t_now, i, n, (time.time()-t)/60 ,calc_eta))
            uid_pid = sp.search(q=txt, type='playlist', limit=p_limit, offset=i,market='US')
            uid_pid_raw.append(uid_pid)
    # Save the data
    with open(data_path, 'wb') as f:
        pickle.dump(uid_pid_raw, f)
        
print(len(uid_pid_raw))

# 2. Get the details of each playlist

In [ ]:
# extract = True
extract = False

# Get the detail of all playlists
format = '%d/%m/%Y %H:%M'
t = time.time()

n = len(uid_pid_raw)
data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_N.pkl'
if extract == True:
    print('Details of each playlist - Data requested from API - Saved:' + data_path)
    # Get details of all playlists
    playlists_full = []
    print('Getting details of all playlists')
    for i, uid_pid in enumerate(uid_pid_raw):
        if i % 100 == 0 and i > 0:
            playlists_full = []
            t_now = datetime.datetime.now().strftime(format)
            calc_eta = datetime.datetime.now() + datetime.timedelta(seconds = n - i) * (time.time() - t) / (i + 1)
            calc_eta = calc_eta.strftime(format)
            print("%s - %d/%d elapsed time (min): %.1f - ETA: %s" % (t_now, i, n, (time.time()-t)/60 ,calc_eta))

        for user in uid_pid['playlists']['items']:
            pid = user['id']
            uid = user['owner']['id']
            playlist = sp.user_playlist(uid, pid)
            playlists_full.append(playlist)

        # Save the data
        if i % 100 == 0 and i > 0:
            print(i)
            data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_' + str(int(i/100)) + '.pkl'
            print(data_path)
            with open(data_path, 'wb') as f:
                pickle.dump(playlists_full, f)

# 3. Concatenate all playlists

## 3.1 Raw playlists

In [4]:
# Concatenate all playlists
format = '%d/%m/%Y %H:%M'
t = time.time()

data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_full.pkl'

if os.path.exists(data_path):
    print('All playlists concatenated - Loaded:' + data_path)
    with open(data_path, 'rb') as f:
        playlists_full_by_user = pickle.load(f)
else:
    print('All playlists concatenated - Saved:' + data_path)
    playlists_full_by_user = []
    for i in range(0,100,1):
        data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_' + str(i) + '.pkl'
        if os.path.exists(data_path):
            print(data_path)
            with open(data_path, 'rb') as f:
                playlist = pickle.load(f)
            playlists_full_by_user.extend(playlist)
            
    data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_full.pkl'
    print('Saving:' + data_path)
    with open(data_path, 'wb') as f:
        pickle.dump(playlists_full_by_user, f)

print(len(playlists_full_by_user))

All playlists concatenated - Loaded:C:\Users\map_f\OneDrive\Documents\GitHub\Follow_the_beat\data\external\raw_2_playlists_full.pkl
2180


## 3.2 Create a dictionary for each playlist

In [104]:
def get_artists(raw_artist):
    artists = []
    artists_id = []
    for a in raw_artist:
        artists.append(a['name'])
        artists_id.append(a['id'])
#     artists.append(artist)
#     artists_id.append(artist_id)
    
    return(artists_id, artists)


# m = playlists_full_by_user[83]
# n = m['tracks']['items']
# p = n[2]['track']['album']['artists']
# print(p)

# a = get_artists(p)
# a

In [248]:
def get_track_details(raw_track):
    if raw_track is not None:
        artists, artists_id = get_artists(raw_track['track']['album']['artists'])
        track = raw_track['track']
        data = {
                'added_at': raw_track['added_at'],
                'track_id': track['id'],
                'track': track['name'],
                'popularity': track['popularity'],

                'albums_id': track['album']['id'],
                'albums': track['album']['name'],
                'release_date': track['album']['release_date'],

                'artists_id': artists_id,
                'artists': artists
            }
    return(data)

# m = playlists_full_by_user[83]
# n = m['tracks']['items'][0]
# n
# a = get_track_details(n)
# a

In [5]:
def get_playlist_detail(playlist):
    artists = []
    albums = []
    tracks = []
    duration = []
    popularity = []
    artists_id = []
    albums_id = []
    tracks_id = []
    release_date = []
    added_at = []
    
#     playlist = sp.user_playlist(user_id, playlist_id)
    
    user_id = playlist['owner']['id']
    playlist_id = playlist['id']
    total_tracks = len(playlist['tracks'])

    for item in playlist['tracks']['items']:
        group = []
        group_id = []
        if item['track'] is not None:
            for artist in item['track']['album']['artists']:
                group.append(artist['name'])
                group_id.append(artist['id'])
            artists.append(group)
            artists_id.append(group_id)

            albums_id.append(item['track']['album']['id'])
            albums.append(item['track']['album']['name'])

            tracks_id.append(item['track']['id'])
            tracks.append(item['track']['name'])
            release_date.append(item['track']['album']['release_date'])
            added_at.append(item['added_at'])
            popularity.append(item['track']['popularity'])

            data = {
                    'user_id': user_id,
                    'playlist_id': playlist_id,
                    'total_tracks': total_tracks,
                    'tracks_id': tracks_id,
                    'tracks': tracks,
                    'popularity': popularity,
                    'artists_id': artists_id,
                    'artists': artists,
                    'albums_id': albums_id,
                    'albums': albums,
                    'release_date': release_date,
                    'added_at': added_at
                }
        
        return data;
# t = get_playlist_detail(playlists_full_by_user[0])
# t

In [ ]:
# playlist = sp.user_playlist('devontay1029', '7AU3xd5nGS3JT5ZVNcXBsM')
# playlist

In [ ]:
# a = playlists_full_by_user[0]['tracks']['items'][0]
# a

In [ ]:
# t.get('playlist_id')

In [ ]:
# for i, playlist in enumerate(playlists_full_by_user):
#     if i ==1:
# #         print(playlist)
#         for a in playlist:
#             print(a)

In [ ]:
play_dict = get_playlist_detail(playlists_full_by_user[6])

In [ ]:
play_dict

In [ ]:
playlists_dict = {}
for i, playlist in enumerate(playlists_full_by_user):
    if i < 10:
        print(i)
        print(playlist['id'])
        play_dict = get_playlist_detail(playlist)
    
        playlists_dict.update({playlist['id']: play_dict})

## 3.3 Create a dictionary for each track

In [ ]:
# s_tracks_id = single_list(df['tracks_id'])
# s_tracks = single_list(df['tracks'])
# s_popularity = single_list(df['popularity'])

# s_artists = single_list_nested(df['artists'])
# s_artists_id = single_list_nested(df['artists_id'])

# s_albums = single_list(df['albums'])
# s_albums_id = single_list(df['albums_id'])

# data = {
#     'tracks_id': s_tracks_id,
#     'tracks': s_tracks,
#     'popularity': s_popularity
# }
# df_track = pd.DataFrame(data)
# # print(df_track.head())

# data = {
#     'artists_id': s_artists_id,
#     'artists': s_artists
# }
# df_artist = pd.DataFrame(data)
# # print(df_artist.head())


# data = {
#     'albums_id': s_albums_id,
#     'albums': s_albums
# }
# df_album = pd.DataFrame(data)
# # print(df_album.head())

In [180]:
# tracks_rank = df_track.groupby(['tracks_id','tracks']).agg(
#     popularity=('popularity','max'),
#     tracks_count=('tracks_id','count')
#     )

# min_max = MinMaxScaler()
# tracks_rank['count_scaled'] = min_max.fit_transform(tracks_rank[["tracks_count"]])

# tracks_rank["rank"] = tracks_rank['tracks_count'].rank(method='average',ascending=False)

# tracks_rank = tracks_rank.sort_values('tracks_count',ascending=False)

# tracks_rank

In [181]:
# album_rank = df_album.groupby(['albums_id','albums']).agg(album_count=('albums_id','count'))

# min_max = MinMaxScaler()
# album_rank['count_scaled'] = min_max.fit_transform(album_rank[["album_count"]])

# album_rank["rank"] = album_rank['album_count'].rank(method='average',ascending=False)

# album_rank = album_rank.sort_values('album_count',ascending=False)

# album_rank

In [182]:
# df_artist.head
# artist_rank = df_artist.groupby(['artists_id','artists']).agg(artist_count=('artists_id','count'))

# min_max = MinMaxScaler()
# artist_rank['count_scaled'] = min_max.fit_transform(artist_rank[["artist_count"]])

# artist_rank["rank"] = artist_rank['artist_count'].rank(method='average',ascending=False)

# artist_rank = artist_rank.sort_values('artist_count',ascending=False)

# artist_rank

In [183]:
# tracks_cartesian = []
# for element in itertools.product(s_tracks_id,s_tracks_id):
#     tracks_cartesian.append(element)

# https://stackoverflow.com/questions/11144513/cartesian-product-of-x-and-y-array-points-into-single-array-of-2d-points
# def cartesian_product(*arrays):
#     la = len(arrays)
#     dtype = np.result_type(*arrays)
#     arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
#     for i, a in enumerate(numpy.ix_(*arrays)):
#         arr[...,i] = a
#     return arr.reshape(-1, la)


# # tracks_cartesian = cartesian_product(*(s_tracks_id * 2))

In [184]:
# for key, item in df_test.items():
#     print(key)
#     if key == 'tracks_id':
#         print(item)

In [ ]:
# # Define count_entries()
# def count_entries(csv_file,c_size,colname):
#     """Return a dictionary with counts of
#     occurrences as value for each key."""
    
#     # Initialize an empty dictionary: counts_dict
#     counts_dict = {}

#     # Iterate over the file chunk by chunk
#     for chunk in pd.read_csv(csv_file,chunksize=c_size):

#         # Iterate over the column in DataFrame
#         for entry in chunk[colname]:
#             if entry in counts_dict.keys():
#                 counts_dict[entry] += 1
#             else:
#                 counts_dict[entry] = 1

#     # Return counts_dict
#     return counts_dict

# # Call count_entries(): result_counts
# result_counts = count_entries('tweets.csv',10,'lang')

# # Print result_counts
# print(result_counts)


In [88]:
# https://www.thedataschool.co.uk/penny-richmond/howtogetdataoffspotify
def analyze_playlist(user_id, playlist_id):

    # Create empty dataframe
    playlist_features_list = ["artist", "album", "track_name", "track_id", "danceability", "energy", "key", "loudness","mode", "speechiness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]

    playlist_df = pd.DataFrame(columns = playlist_features_list)

    # Loop through every track in the playlist, extract features and append the features to the playlist df

    playlist = sp.user_playlist_tracks(user_id, playlist_id)["tracks"]["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]

    # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]

        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    return playlist_df

In [95]:
audio_features = sp.audio_features('0LyfQWJT6nXafLPZqxe9Of')
audio_features

[None]

In [199]:
# import GSA
# myPlaylist = GSA.getInformation('37i9dQZF1DX3hgbB9nrEB1', verbose=True)

In [102]:
# df = analyze_playlist('12158094445','3whqQJWBWirCJW1EeeMw5V')
# df

In [231]:
a = playlists_full_by_user[0]
pid = a['id']
owner = a['owner']
tracks = a['tracks']

track = tracks['items'][0]['track']['album']
track
# type(track[0])
# artist =track[0]
for i, feat in enumerate(track):
    print(feat)
#     print(type(feat))
#     print()

album_type
artists
available_markets
external_urls
href
id
images
name
release_date
release_date_precision
total_tracks
type
uri


In [138]:
m = playlists_full_by_user[83]
# m
# n = m['tracks']['items']
# n
p = n[2]['track']
p['']['name']

# a = playlists_full_by_user[6]['tracks']['items']
# a[0]['track']['album']['artists']
# for i in range(83,200,1):
#     print (i)
#     print(a[i]['track']['album']['artists'])


# for a in m:
#     print(a)
# m['items']
# p = n[2]['track']['album']['artists']
# print(p)




'Nowadays (feat. Landon Cube)'